# **Extracción y limpieza de datos en Spotify**

In [3]:
# Hacemos las importaciones necesarias para que funcione spotipy y sean correctas las llamadas y autenticaciones
import spotipy
from spotipy import Spotify
from spotipy.oauth2 import SpotifyClientCredentials
# Para poder leer los ficheros csv y hacer DataFrames
import pandas as pd
# Para esconder la API key
import os 
# Para esconder la API key
from dotenv import load_dotenv

In [17]:
# Leer el archivo env, que tiene las claves - pq las tenemos ocultas y lo guardamos en una variable
load_dotenv()
client_secret = os.getenv("api_key_spotify")

In [18]:
# URL para bajar los datos. Nos hemos dado de alta como cliente y nos han dado un id y una clave que tenemos oculta
url = "https://accounts.spotify.com/api/token"

client_id = "54778a9f1fd7438d9c5710a72d0ca202"

# Autenticación

client_credential = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)

sp = spotipy.Spotify(auth_manager=client_credential)

In [ ]:
# Hacemos la llamada mediante una función, ya que tenemos que pedir datos por género y año 

def extraccion_datos(años, generos):
    datos = []
    for año in años:
        for genero in generos:
            # con offset paginamos los resultados porque la API no deja sacar mas de 50 datos a la vez
            for offset in range(0, 1000, 50): # Decidimos sacar 1000 datos por cada género y año 
                # Aquí hacemos la llamada y guardamos los datos en datos, para su posterior limpieza
                datos.append(sp.search(q=f"genre:{genero} year:{año}", type='track', limit=50, offset=offset))
    return datos

In [ ]:
# Usamos listas para suministrar los parámetros a la función. Pudiendo elegir entre uno o varios años y entre uno o varios géneros
años = ['2021', '2022']
generos = ['pop', 'rock', 'indie', 'rap']

In [ ]:
# Llamada a la función para la extracción de datos, con los parámetros que le hemos indicado en las tablas de arriba
info = extraccion_datos(años, generos)

In [ ]:
# Limpieza de datos
  
def filtrado_datos(datos, generos): 
    # Recogemos los datos filtrados en un diccionario con los campos exigidos
    datos_filtrados = {"artista": [], "tipo": [], "título_canción": [], "nombre_album": [], "año": [], 'genre': []}
    # Iteramos por cada elemento recogido en datos para sacar los resultados que necesitamos. 
        # Sabemos su número de datos haciendo un len
    # Descubrimos que todos los datos que necesitábamos estaban en la lista  tracks-items , excepto género
    for i in range(0, len(datos)): 
        for elemento in datos[i]["tracks"]["items"]:
            datos_filtrados["artista"].append(elemento["album"]["artists"][0]["name"])
            datos_filtrados["tipo"].append(elemento['type'])
            datos_filtrados["título_canción"].append(elemento['name'])
            datos_filtrados["nombre_album"].append(elemento["album"]["name"])
            datos_filtrados["año"].append(elemento["album"]["release_date"][0:4])
            # Al ser un dato que suministramos como parámetro en la función de extracción, solo se encuentra en 'href'
            for genero in generos: # Buscamos el género comparádolo con la lista género usada como parámetro
                # Iteramos por cada uno de esos géneros en datos y si encuentra coincidencia, lo agrega como valor al diccionario
                if genero in datos[i]["tracks"]['href']:
                    datos_filtrados['genre'].append(genero)
    # Hacemos un DataFrame con los valores del diccionario creado con el filtrado de datos    
    df_concatenado =pd.DataFrame(datos_filtrados)
    # Y lo convertimos en un csv que posteriormente usaremos en la carga de datos en SQL
    df_concatenado.to_csv('spotify_df_concatenado.csv', index=False)

    # Aprovechamos este csv para hacer una lista de artistas únicos, 
    # que van a ser la base de las extracciones de las API's MusicBrainz y LastFM
    df_spotify = pd.read_csv('spotify_df_concatenado.csv')
    lista_artistas = list(df_spotify['artista'].unique())
    # Hacemos un DataFrame con esta lista de artistas únicos
    df_lista_artistas = pd.DataFrame(lista_artistas)
    # Para poder hacer un DataFrame que podamos usar en las otras extracciones
    df_lista_artistas.to_csv('lista_artistas_spotify.csv', index=False)

    return df_concatenado

In [24]:
# Llamamos a la función que filtra los datos
data = filtrado_datos(info, generos)

In [25]:
data.head()

,artista,tipo,título_canción,nombre_album,año,genre
0,Tory Lanez,track,The Color Violet,Alone At Prom,2021,pop
1,PinkPantheress,track,Pain,to hell with it,2021,pop
2,Eyedress,track,Something About You,Mulholland Drive,2021,pop
3,Jordana,track,Better in the Dark,Summer's Over,2021,pop
4,Tate McRae,track,you broke me first,TOO YOUNG TO BE SAD,2021,pop


Las siguientes extraciones son ejemplos de cómo las funciones sirven para cualquier año/s o artista/s que se le proporcione como parámetros

In [12]:
info2 = extraccion_datos(['2020'], ['jazz'])

In [14]:
data2 = filtrado_datos(info2, ['jazz'])

In [16]:
data2.head()

,artista,tipo,título_canción,nombre_album,año,genre
0,Underground Canopy,track,Feelm Intermission,Bluestaeb and S. Fidelity Present Underground ...,2020,jazz
1,Vitamin String Quartet,track,Strange (feat. Hillary Smith),Bridgerton (Covers from the Netflix Original S...,2020,jazz
2,Various Artists,track,It's All Right,Soul (Original Motion Picture Soundtrack),2020,jazz
3,Tom Misch,track,Nightrider,What Kinda Music,2020,jazz
4,Erroll Garner,track,Misty - Remastered 2020,Gemini (Octave Remastered Series),2020,jazz
